On va essayer de clenner les fichier par fichier et voir si y'a moyen d'améliorer ca

In [19]:
import pandas as pd
import numpy as np
import sys
import glob
import os
import math
import dask.dataframe as dd
import re
import heapq
import numpy as np
import shutil
import pyarrow.parquet as pq


In [44]:
parquet_folder = os.path.join(sys.path[0], "data_parquet")
parquet_folder_tracks = os.path.join(sys.path[0], "data_tracks_parquet")

all_files = glob.glob(os.path.join(parquet_folder, "*.parquet"))
all_track_files = glob.glob(os.path.join(parquet_folder_tracks, "*.parquet"))

# Trier les fichiers par le numéro de début de slice
def sort_key(f):
    m = re.search(r"mpd\.slice\.(\d+)-\d+\.parquet", f)
    return int(m.group(1)) if m else float('inf')

all_track_files = sorted(all_track_files, key=sort_key)

On fait un groupage intra fichier

In [ ]:

for i, file in enumerate(all_track_files):
    df = pd.read_parquet(file)
    # groupby local uniquement dans ce fichier
    grouped = (
            df.groupby("track_id").agg({
            "pid": list,             # fusionne la liste de pid
            "track_name": "first",   # garde la première occurrence
            "artist_name": "first"   # idem
        }).reset_index()
    )
    output_file = f"tracks_grouped/grouped_{i:04d}.parquet"
    grouped.to_parquet(output_file, index=False)
    if (i+1) % 100 == 0:
        print(f"Processed {i+1} files")


Processed 0 files
Processed 100 files
Processed 200 files
Processed 300 files
Processed 400 files
Processed 500 files
Processed 600 files
Processed 700 files
Processed 800 files
Processed 900 files


On va trier les musique intra fichier

In [48]:
import pyarrow.parquet as pq
import pyarrow as pa
import os

INPUT_DIR = "tracks_grouped"
TEMP_DIR = "temp_chunks"
os.makedirs(TEMP_DIR, exist_ok=True)

CHUNK_SIZE = 100_000  # nombre de lignes à traiter en mémoire par chunk

chunk_id = 0
for filename in os.listdir(INPUT_DIR):
    if not filename.endswith(".parquet"):
        continue
    table = pq.read_table(os.path.join(INPUT_DIR, filename))
    df = table.to_pandas()

    # Diviser en chunks si fichier très gros
    for start in range(0, len(df), CHUNK_SIZE):
        chunk = df.iloc[start:start + CHUNK_SIZE]
        # Tri en mémoire par track_id
        chunk = chunk.sort_values("track_id")
        # Sauvegarder chunk trié
        chunk_file = os.path.join(TEMP_DIR, f"chunk_{chunk_id:04d}.parquet")
        chunk.to_parquet(chunk_file, index=False)
        chunk_id += 1

print(f"{chunk_id} chunks créés dans {TEMP_DIR}")


1000 chunks créés dans temp_chunks


On trie les musiques interfichier

In [ ]:
TEMP_DIR = "temp_chunks"
OUTPUT_DIR = "track_vectors"
os.makedirs(OUTPUT_DIR, exist_ok=True)

LINES_PER_FILE = 15_000  # lignes par fichier final
BUFFER_SIZE = 15_000       # écrire par lot pour performance

def parquet_row_generator(filename):
    for batch in pq.read_table(filename).to_batches():
        df = batch.to_pandas()
        for _, row in df.iterrows():
            yield row

chunk_files = [os.path.join(TEMP_DIR, f) for f in os.listdir(TEMP_DIR)]
generators = [parquet_row_generator(f) for f in chunk_files]
merged_iter = heapq.merge(*generators, key=lambda r: r["track_id"])

file_idx = 0
lines_in_file = 0
buffer = []

current_id = None
current_list = []

for row in merged_iter:
    tid = row["track_id"]
    lst = row["pid"] 
    art = row["artist_name"]
    name = row["track_name"] 
    if isinstance(lst, np.ndarray):
        lst = lst.tolist()

    if current_id is None:
        current_id = tid
        current_list = lst
        current_artist = art
        current_name = name
    elif tid == current_id:
        current_list.extend(lst)
    else:
        buffer.append({"track_id": current_id, "artist_name": current_artist, "track_name": current_name, "pid": current_list})
        lines_in_file += 1

        current_id = tid
        current_list = lst
        current_artist = art
        current_name = name
    if len(buffer) >= BUFFER_SIZE:
        file_path = os.path.join(OUTPUT_DIR, f"track_vector_{file_idx:04d}.parquet")
        if os.path.exists(file_path):
            # si le fichier existe déjà, créer un nouveau fichier
            file_idx += 1
            lines_in_file = 0
            file_path = os.path.join(OUTPUT_DIR, f"track_vector_{file_idx:04d}.parquet")
        table = pa.Table.from_pandas(pd.DataFrame(buffer))
        pq.write_table(table, file_path)
        buffer = []

    # changer de fichier si LINES_PER_FILE atteint
    if lines_in_file >= LINES_PER_FILE:
        file_idx += 1
        lines_in_file = 0

# écrire la dernière ligne
if current_id is not None:
    buffer.append({"track_id": current_id, "artist_name": current_artist, "track_name": current_name, "pid": current_list})

# écrire ce qui reste
if buffer:
    file_path = os.path.join(OUTPUT_DIR, f"track_vector_{file_idx:04d}.parquet")
    table = pa.Table.from_pandas(pd.DataFrame(buffer))
    pq.write_table(table, file_path)

print(f"Partitionnement terminé dans {OUTPUT_DIR}")


Partitionnement terminé dans track_vectors


Suprimes les dossiers intermédiaires

In [16]:
folder = os.path.join(sys.path[0], "temp_chunks")
if os.path.exists(folder):
    shutil.rmtree(folder)

folder = os.path.join(sys.path[0], "tracks_grouped")
if os.path.exists(folder):
    shutil.rmtree(folder)


Charge les fichiers track_vectors

In [3]:
parquet_folder_tracks = os.path.join(sys.path[0], "track_vectors")
df_tracks_vectors = dd.read_parquet(parquet_folder_tracks)
df_tracks_vectors = df_tracks_vectors.set_index("track_id", sorted= True)

On construit une indexation ce qui permet d'utiliser panda pour aller chercher seulement la db dont on a besoin

In [ ]:
# index tous les fichiers track_vectors pour connaitre rapidement dans quel fichier chercher un track_id
def build_index(folder):
    index = []
    
    for filename in sorted(os.listdir(folder)):
        if not filename.endswith(".parquet"):
            continue
        
        path = os.path.join(folder, filename)
        
        # lire uniquement le row_group 0 (hyper rapide)
        table = pq.read_table(path, columns=["track_id"], read_dictionary=["track_id"])
        col = table.column("track_id")
        
        min_id = col[0].as_py()
        max_id = col[-1].as_py()
        index.append((min_id, max_id, filename))
    
    return index
INDEX = build_index(os.path.join(sys.path[0],"track_vectors"))

Permet d'utiliser panda et d'ouvrir le fichier dans lequel se trouve le track id

In [42]:
def find_file_for_track_id(track_id):
    low, high = 0, len(INDEX) - 1
    
    while low <= high:
        mid = (low + high) // 2
        min_id, max_id, filename = INDEX[mid]
        
        if track_id < min_id:
            high = mid - 1
        elif track_id > max_id:
            low = mid + 1
        else:
            return os.path.join(sys.path[0], "track_vectors", filename)
    return None

find_file_for_track_id('3Y6ILqXqCBNwBTxVda8Z68')


'c:\\Users\\ryanm\\Documents\\master big data\\info\\projet\\track_vectors\\track_vector_0068.parquet'

In [44]:
subset1 = df_tracks_vectors.loc[['3Y6ILqXqCBNwBTxVda8Z68','3Y6ILqXqCBNwBTxVda8Z68'], "pid"].compute()

In [62]:
def get_rows_in_order(df, track_ids):
    # un seul fetch Dask → compute()
    unique_ids = list(dict.fromkeys(track_ids))
    tmp = df.loc[unique_ids,'pid'].compute()

    # reconstruction dans l'ordre demandé
    rows = [tmp.loc[tid] for tid in track_ids]

    return rows
row = get_rows_in_order(df_tracks_vectors, ['3Y6ILqXqCBNwBTxVda8Z68','09bLMpxeUECNq4v5BIdgp6','3Y6ILqXqCBNwBTxVda8Z68'])

Ici c'est la fonction principale

In [ ]:
# Cette fonction évalue la similarité entre deux pistes en fonction des playlists dans lesquelles elles apparaissent.
# De plus on peut lui fournir une liste de pistes ce qui permet immédiatement de résoudre la question 4
def similar_tracks(track_id1, track_id2, which = 0):
    ## partie si on donne une liste d'id, typiquement la partie pour l'évaluation de playlist
    print(which)
    if isinstance(track_id1, list):
        ## calcul de zoé sans poids
        try:
        # version un peu moins naive avec un seul appel à dask
        # on peut faire mieux en utilisant pandas pour reconstruire les listes dans l'ordre demandé
            unique_ids = list(dict.fromkeys(track_id1+ track_id2))
            tmp = df_tracks_vectors.loc[unique_ids,'pid'].compute()
            # reconstruction dans l'ordre demandé
            rows = [tmp.loc[tid] for tid in track_id1 + track_id2] 
            list1 = [x for sub in rows[0:len(track_id1)] for x in sub]
            list2 = [x for sub in rows[len(track_id1):] for x in sub]         
        #Version très naive
        # ici on peut optimiser en récupérant les deux listes en une seule requête ca va diviser le temps par 2
        #try:
        #    list1 = df_tracks_vectors.loc[track_id1, "pid"].compute().explode().tolist()
        #    list2 = df_tracks_vectors.loc[track_id2, "pid"].compute().explode().tolist()
        except KeyError:
            print("Le numéro est out of range")
            return None
    else:
        ## partie où on utilise un seul track_id. On fait le calcul avec panda
        # on pourrai aussi optimiser si les deux musique sont dans le même fichier mais bon ca à a 1/150 chance d'arriver donc
        # flemme d'optimiser pour si peu
        try:
            # Récupérer les colonnes "list_pid"
            df = pd.read_parquet(find_file_for_track_id(track_id1),
                     columns=["track_id", "pid"])
            list1 = df.loc[df.track_id == track_id1, "pid"].iloc[0]
            df = pd.read_parquet(find_file_for_track_id(track_id2),
                     columns=["track_id", "pid"])
            list2 = df.loc[df.track_id == track_id2, "pid"].iloc[0]
            which = 0     # le calcul des autre ne prend du sens que si c'est une liste
        except KeyError:
            print("string")
            return None
    # transformer en sets pour intersection
    set1, set2 = set(list1), set(list2)
    n_common = len(set1 & set2)
    res_main = n_common / math.sqrt(len(set1) * len(set2))
    if which == 0:
        if n_common == 0:
            return 0.0
        return res_main
    else:
        res_zoe = len(set(track_id1) & set(track_id2)) / math.sqrt(len(track_id1) * len(track_id2))
        res_sup = 0.0
        print("qqch")
        for l1 in rows[0:len(track_id1)]:
            sim = 0
            for l2 in rows[len(track_id1):]:
                sim = max(sim, len(set(l1) & set(l2)) /math.sqrt(len(l1) * len(l2) ))
                if sim == 1.0:
                    break
            res_sup = res_sup + sim
        res_sup = res_sup / (len(track_id1)*len(track_id2))**0.5
        return (res_main, res_zoe, res_sup)

In [104]:
print(similar_tracks(get_track_ids_for_pid(78561),get_track_ids_for_pid(235921),1))
print(similar_tracks(get_track_ids_for_pid(2000),get_track_ids_for_pid(2000),1))

1
qqch
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
(0.1998485668390489, 0.0, 0.02787419942619766)
1
qqch
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
(1.0, 1.0, 0.9875803727323389)


In [ ]:
def get_track_ids_for_pid(n: int):
    if not (0 <= n <= 10**6):
        raise ValueError("n doit être entre 0 et 1_000_000 inclus.")
    # Calcul du range
    x = (n // 1000) * 1000
    y = x + 999
    filename = f"mpd.slice.{x}-{y}.parquet"
    # Lecture du fichier parquet
    filepath = os.path.join(sys.path[0], "data_tracks_parquet", filename)
    df = pd.read_parquet(filepath)

    # Filtrer les lignes où pid == n
    filtered = df[df["pid"] == n]

    # Retourner la colonne track_id comme liste
    return filtered["track_id"].tolist()

def get_file_for_track_id(track_id):
    

In [37]:
similar_tracks('3Y6ILqXqCBNwBTxVda8Z68','3Y6ILqXqCBNwBTxVda8Z68')


1.0

In [ ]:
print(similar_tracks(get_track_ids_for_pid(78561),get_track_ids_for_pid(235921)))
print(similar_tracks(get_track_ids_for_pid(2000),get_track_ids_for_pid(2000)))


0.1998485668390489

In [88]:
print(similar_tracks(get_track_ids_for_pid(78561),get_track_ids_for_pid(235921)),1)
print(similar_tracks(get_track_ids_for_pid(2000),get_track_ids_for_pid(2000)),1)


0.1998485668390489 1
1.0 1


In [73]:
track_id1 = get_track_ids_for_pid(78561)
track_id2 = get_track_ids_for_pid(235921)
unique_ids = list(dict.fromkeys(track_id1+ track_id2))
tmp = df_tracks_vectors.loc[unique_ids,'pid'].compute()
# reconstruction dans l'ordre demandé
rows = [tmp.loc[tid] for tid in track_id1 + track_id2] 
list1 = [x for sub in rows[0:len(track_id1)] for x in sub]
list2 = [x for sub in rows[len(track_id1):] for x in sub] 

In [74]:
list2

[np.int64(327),
 np.int64(344),
 np.int64(927),
 np.int64(1062),
 np.int64(1255),
 np.int64(1425),
 np.int64(1726),
 np.int64(2424),
 np.int64(3068),
 np.int64(3074),
 np.int64(3158),
 np.int64(3599),
 np.int64(3729),
 np.int64(4382),
 np.int64(4706),
 np.int64(5909),
 np.int64(6701),
 np.int64(6766),
 np.int64(6998),
 np.int64(7326),
 np.int64(7813),
 np.int64(8492),
 np.int64(8542),
 np.int64(8632),
 np.int64(8656),
 np.int64(9044),
 np.int64(9261),
 np.int64(9304),
 np.int64(10206),
 np.int64(11053),
 np.int64(11471),
 np.int64(11772),
 np.int64(11993),
 np.int64(12009),
 np.int64(12032),
 np.int64(12302),
 np.int64(12505),
 np.int64(12895),
 np.int64(13098),
 np.int64(13270),
 np.int64(13653),
 np.int64(13983),
 np.int64(14528),
 np.int64(15786),
 np.int64(15790),
 np.int64(16053),
 np.int64(16106),
 np.int64(16257),
 np.int64(16399),
 np.int64(16681),
 np.int64(18053),
 np.int64(18075),
 np.int64(18159),
 np.int64(18402),
 np.int64(18540),
 np.int64(18621),
 np.int64(18913),
 np.i

In [80]:
len(set(get_track_ids_for_pid(96284))&set(get_track_ids_for_pid(96284)))/ ((len(get_track_ids_for_pid(96284)) * len(get_track_ids_for_pid(96284)))**0.5) 

1.0